## Load the Data

In [12]:
import numpy as np
import pandas as pd
from Korpora import Korpora

# 테스트 데이터셋 20000개의 학습 데이터에 대해서 학습 진행
corpus = Korpora.load("nsmc")
df = pd.DataFrame(corpus.test).sample(20000, random_state=42)

# train : val : test 를 6 : 2 : 2 로 분리
train_df, val_df, test_df = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))]
)

# 출력 확인
print(train_df.head(5).to_markdown())
print(f"train: {len(train_df)}, val: {len(val_df)}")
print(f"test: {len(test_df)}")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/ho/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /home/ho/Korpora/n

## Data Pipeline

In [14]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text = data.text.to_list(),
        padding="longest", # 배치 내에서 가장 긴 sequence 길이로 통일
        truncation=True, # 모델이 처리할 수 있는 최대 길이로 입력 조정 (BERT의 경우, 512)
        return_tensors="pt"
    )

    input_ids = tokenized["input_ids"].to(device)
    att_masks = tokenized["attention_mask"].to(device)
    labels = torch.tensor(data.label.values, dtype=torch.long).to(device)
    
    return TensorDataset(input_ids, att_masks, labels)

def get_dataloader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    data_loader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)

    return data_loader

## Network Architecture

In [25]:
def check_network(net):
    for main_name, main_module in net.named_children():
        print(main_name)

        for sub_name, sub_module in main_module.named_children():
            print("- ", sub_name)

            for ssub_name, ssub_module in sub_module.named_children():
                print("|    - ", ssub_name)

                for sssub_name, _ in ssub_module.named_children():
                    print("|    |   - ", sssub_name)
    
    return sub_name

## Train Pipeline

In [15]:
import numpy as np
import torch.nn as nn

def calc_accuracy(preds, labels):
    pred_flatten = np.argmax(preds, axis=1).flatten()
    labels_flatten = labels.flatten()

    return np.sum(pred_flatten == labels_flatten) / len(labels_flatten)

def train(net, optimizer, data_loader):
    net.train()
    train_loss = 0.0

    for input_ids, att_mask, labels in data_loader:
        optimizer.zero_grad()

        outputs = net(
            input_ids=input_ids,
            attention_mask=att_mask,
            labels=labels
        )

        loss = outputs.loss
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    train_loss /= len(data_loader)
    return train_loss

def eval(net, data_loader):
    net.eval()
    criterion = nn.CrossEntropyLoss()
    val_loss, val_acc = 0.0, 0.0

    with torch.no_grad():
        for input_ids, att_mask, labels in data_loader:
            outputs = net(
                input_ids=input_ids,
                attention_mask=att_mask,
                labels=labels
            )
            
            logits = outputs.logits
            loss = criterion(logits, labels)
            
            logits.detach().cpu().numpy()
            labels = labels.to("cpu").numpy()
            acc = calc_accuracy(logits, labels)

            val_loss += loss
            val_acc += acc

    val_loss /= len(data_loader)
    val_acc /= len(data_loader)

    return val_loss, val_acc

## Hyper-parameter

In [30]:
EPOCHS = 5
BATCH_SIZE = 32
LR = 1e-5
EPS = 1e-8

device = "cuda" if torch.cuda.is_available() else "cpu"

## Self-Supervised Learning

In [27]:
from transformers import AutoTokenizer
from torch.utils.data import RandomSampler, SequentialSampler

def prepare_data(pretrained):
    tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=pretrained,
        do_lower_case=False
    )

    train_dataset = make_dataset(train_df, tokenizer, device)
    train_loader = get_dataloader(train_dataset, RandomSampler, BATCH_SIZE)

    val_dataset = make_dataset(val_df, tokenizer, device)
    val_loader = get_dataloader(val_dataset, SequentialSampler, BATCH_SIZE)

    test_dataset = make_dataset(test_df, tokenizer, device)
    test_loader = get_dataloader(test_dataset, SequentialSampler, BATCH_SIZE)

    return train_loader, val_loader, test_loader

In [28]:
from transformers import AutoModelForSequenceClassification

def get_network(pretrained):
    net = AutoModelForSequenceClassification.from_pretrained(
        pretrained, num_labels=2
    ).to(device)

    return net

In [31]:
from torch.optim import AdamW

def main(pretrained, optimizer):
    train_loader, val_loader, test_loader = prepare_data()
    net = get_network(pretrained)
    optimizer = AdamW(net.parameters(), lr=LR, eps=EPS)

    # 아키텍처 구조 확인
    sub_name = check_network(net)

    # 훈련
    best_loss = 1e9
    for epoch in range(EPOCHS):
        train_loss = train(net, optimizer, train_loader)
        val_loss, val_acc = eval(net, val_loader)

        print(f"Epoch [{epoch+1}/{EPOCHS}]")
        print(f"  Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(net.state_dict(), "./pth/{sub_name}.pth")

In [ ]:
# bert
main(pretrained="bert-base-multilingual-cased", optimizer=optimizer)

In [ ]:
# gpt
main(pretrained="")